<a href="https://colab.research.google.com/github/Jorgecardetegit/computer_vision_peripheral_blood/blob/main/training_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
directory_path="/content/drive/MyDrive/peripheral blood dataset"

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)

In [ ]:
train_ds = data_gen.flow_from_directory(
    directory=directory_path,
    subset="training",
    seed=123,
    target_size=(224,224),
    batch_size=32,
    class_mode='sparse',
    shuffle=True
)

val_ds = data_gen.flow_from_directory(
    directory=directory_path,
    subset="validation",
    seed=123,
    target_size=(224,224),
    batch_size=32,
    class_mode='sparse',
    shuffle=True)

Found 13677 images belonging to 8 classes.
Found 3415 images belonging to 8 classes.


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras import metrics

model = models.Sequential([
    layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"),
    layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', metrics.F1Score(), metrics.Precision(), metrics.Recall()])

model.summary()

history = model.fit(
    train_ds,
    epochs=10,  # You can adjust the number of epochs
    validation_data=val_ds
)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import metrics

# Load the ResNet50 model pre-trained on ImageNet, excluding the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax') # Assuming 5 classes
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', metrics.F1Score(), metrics.Precision(), metrics.Recall()])

model.summary()

history = model.fit(
    train_ds,
    epochs=10,  # You can adjust the number of epochs
    validation_data=val_ds
)

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import metrics

# Load the EfficientNetB0 model pre-trained on ImageNet, excluding the top classification layer
# You can specify the input shape here
base_model_efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base EfficientNetB0 model
for layer in base_model_efficientnet.layers:
    layer.trainable = False

# Create a new model using the EfficientNetB0 base
model_efficientnet = models.Sequential([
    base_model_efficientnet,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax') # Assuming 5 classes
])

model_efficientnet.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', metrics.F1Score(), metrics.Precision(), metrics.Recall()])

model_efficientnet.summary()

history_efficientnet = model_efficientnet.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy for ResNet50
plt.plot(history.history['accuracy'], label='ResNet50 Training Accuracy')
plt.plot(history.history['val_accuracy'], label='ResNet50 Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('ResNet50 Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss for ResNet50
plt.plot(history.history['loss'], label='ResNet50 Training Loss')
plt.plot(history.history['val_loss'], label='ResNet50 Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('ResNet50 Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy for EfficientNetB0
plt.plot(history_efficientnet.history['accuracy'], label='EfficientNetB0 Training Accuracy')
plt.plot(history_efficientnet.history['val_accuracy'], label='EfficientNetB0 Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('EfficientNetB0 Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss for EfficientNetB0
plt.plot(history_efficientnet.history['loss'], label='EfficientNetB0 Training Loss')
plt.plot(history_efficientnet.history['val_loss'], label='EfficientNetB0 Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('EfficientNetB0 Training and Validation Loss')
plt.legend()
plt.show()